In [2]:
import os
import pandas as pd
import numpy as np
import datetime

In [5]:
rebalData =  pd.read_excel('190619.xlsx') # 전체 리밸런싱 스케줄 불러오기
priceData = pd.read_excel('PE Ratio_190530.xlsx', sheet_name='P')  # 전체 가격 데이터 불러오기

In [4]:
rebalData.head()

,date,code,weight
0,2001-02-28,A000700,0.033333
1,2001-02-28,A001440,0.033333
2,2001-02-28,A025830,0.033333
3,2001-02-28,A011200,0.033333
4,2001-02-28,A003030,0.033333


In [6]:
priceData.head()

,Refresh,Last Update : 2019-05-30 14:37:20,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1188,Unnamed: 1189,Unnamed: 1190,Unnamed: 1191,Unnamed: 1192,Unnamed: 1193,Unnamed: 1194,Unnamed: 1195,Unnamed: 1196,Unnamed: 1197
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Time Series (Company),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Frequency,M,Ascending,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Period(From),19990101,Korean,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Period(To),CPD [20190530],NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def data_cleansing_quantiwise(rawData):
    '''Quantiwise 제공 시계열데이터 클렌징 용도
    - 열 : 종목명
    - 행 : 시계열
    '''    
    firmCode = rawData.iloc[6, 1:].values
    dateIndex = rawData.iloc[13:, 0].values
    newData = rawData.iloc[13:,1:]
    newData.columns = firmCode
    newData.index = dateIndex
    return newData

In [8]:
priceData = data_cleansing_quantiwise(priceData)
priceData.head()

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A300720,A306200,A307950,A316140,A900030,A900050,A900140,A950010,A950070,A950100
1999-01-29,5823,2640,8700,16333,1413,1076,13933,4500,NaN,14755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-02-26,4570,2219,8200,12737,1298,1076,11557,3360,NaN,13625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03-31,6140,2521,12000,12287,1406,1525,16526,3830,NaN,15608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04-30,7000,3116,10300,13186,1867,1819,21926,5700,NaN,16802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-05-31,6260,3399,7750,14685,1961,1664,24410,4750,NaN,19147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
rebalDate = list(set(rebalData.date.values))
rebalDate.sort()
aaaa = priceData.loc[(rebalDate),:]
aaaa

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A300720,A306200,A307950,A316140,A900030,A900050,A900140,A950010,A950070,A950100
2001-02-28,2545,1661,865,7792,1889,316,11557,3100,NaN,22286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-05-31,2685,1926,865,9141,1817,558,15553,4000,NaN,25596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-08-31,2220,1500,865,9740,1593,413,14219,3600,NaN,34036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-11-30,3330,1940,865,7792,1608,646,16533,3330,NaN,34146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-28,5720,2040,865,4600,1860,1393,27445,3500,NaN,34856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-05-31,6100,1592,NaN,4713,1957,1184,20446,3900,NaN,33815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-08-30,5660,1486,NaN,4713,1931,1099,22595,3900,NaN,30694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-29,4725,1390,NaN,5559,1950,1169,23863,3240,NaN,30752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28,3675,1314,NaN,4675,1836,964,18572,NaN,NaN,30291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-05-30,3860,1520,NaN,1738,2144,1087,22761,NaN,NaN,34653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
aaaa.pct_change()

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A300720,A306200,A307950,A316140,A900030,A900050,A900140,A950010,A950070,A950100
2001-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-05-31,0.055010,0.159542,0.000000,0.173126,-0.038115,0.765823,0.345764,0.290323,NaN,0.148524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-08-31,-0.173184,-0.221184,0.000000,0.065529,-0.123280,-0.259857,-0.085771,-0.100000,NaN,0.329739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-11-30,0.500000,0.293333,0.000000,-0.200000,0.009416,0.564165,0.162740,-0.075000,NaN,0.003232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-28,0.717718,0.051546,0.000000,-0.409651,0.156716,1.156347,0.660013,0.051051,NaN,0.020793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-05-31,0.066434,-0.219608,0.000000,0.024565,0.052151,-0.150036,-0.255019,0.114286,NaN,-0.029866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-08-30,-0.072131,-0.066583,0.000000,0.000000,-0.013286,-0.071791,0.105106,0.000000,NaN,-0.092296,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-29,-0.165194,-0.064603,0.000000,0.179504,0.009839,0.063694,0.056119,-0.169231,NaN,0.001890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28,-0.222222,-0.054676,0.000000,-0.159021,-0.058462,-0.175364,-0.221724,0.000000,NaN,-0.014991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-05-30,0.050340,0.156773,0.000000,-0.628235,0.167756,0.127593,0.225555,0.000000,NaN,0.144003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 가격데이터가 이미 업로드되어 있다는 전제하에 실행가능한 함수

def get_stock_price(stockCodes, date_start, date_end):
    
    
    return stock_price


def get_basket_history(stockCodes, numStock, date_start, date_end):
    basketPriceData =  get_stock_price(stockCodes, date_start, date_end).fillna(0)
    dates = basketPriceData.index
    basket_history = basketPriceData[stockCodes].values.dot(numStock)    
    basket_history = pd.DataFrame(priceHistory, index = dates)
    return basket_history


In [ ]:
dollar_inv = 100
dollar_ongoing = 100
tradeCost = 0.008
basket_history = {}
rebal_date = list(set(rebalData.date))
rebal_date.sort()
tradeCost_history = {}
turnover_history = {}
weight_history = {}
inv_money_list = [dollar_inv]   
inv_money_date = [rebal_date[0]]

In [ ]:
for i in range(len(rebal_date)-1):
   
    stock_list_i = rebalData[rebalData['date'] == rebal_date[i]].code  # 기간별 종목 리스트 
    w_ = rebalData[rebalData.date == rebal_date[i]].weight 
    
    weightData = pd.concat([stock_list_i, w_], axis = 1).set_index('code')        

    if i == 0:
        weightData['money'] = weightData['weight'] * dollar_inv  # 1기인 경우 초기 투자금으로 시작
    else:
        weightData['money'] = weightData['weight'] * dollar_ongoing # 1기 이후는 투자금의 매 기간별 마지막 시점에서의 금액 재투자            

    stock_price_ = get_stock_price(stock_list_i, rebal_date[i], rebal_date[i]).transpose()  # 해당 일자의 바스켓의 주가 추출     

    weightData = pd.merge(weightData, stock_price_, how = 'inner', left_on= weightData.index, 
                          right_on = stock_price_.index).set_index('key_0')

    weightData.columns = ['weight', 'money', 'price']
    weightData['n_stocks'] = weightData['money'] / weightData['price']

    basket_price = basket_history(weightData.index.values, weightData['n_stocks'].values, rebal_date[i], rebal_date[i+1])     

    dollar_ongoing = basket_price.iloc[-1,:].values[0]  # 투자금의 매 기간별 마지막 시점에서의 포트의 가치(금액)

    # 투자기간 마지막 시점 (다음리밸 시점과 동일) 에서의 가격
    last_price = get_stock_price(weightData.index.values, rebal_date[i+1], rebal_date[i+1]).transpose()  

    weightData = pd.merge(weightData, last_price, how = 'inner', 
                          left_on = weightData.index, right_on = last_price.index).set_index('key_0')

    weightData.columns = ['weight', 'money', 'price', 'n_stocks', 'last_price']

    weightData['last_weight'] = (weightData['last_price'] * weightData['n_stocks']) / (weightData['last_price'] * weightData['n_stocks']).sum()  # 투자기간의 마지막 시점에서의 비중         
    weightData['new_weight'] = rebalData[rebalData['date'] == rebal_date[i+1]].set_index('code')['weight']  # 새로운 비중            
    tradingCost = np.abs(weightData['new_weight'] - weightData['last_weight']).sum() * tradeCost * dollar_ongoing

    dollar_ongoing -= tradingCost     

    inv_money_list += list(basket_price.values.squeeze()[1:-1])
    inv_money_list += [dollar_ongoing]

    inv_money_date += list(basket_price.index[1:])  
    
    basket_history[rebalDate] = basket_price
    tradeCost_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = tradingCost
    turnover_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = tradingCost / dollar_ongoing
    weight_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = weightData


In [ ]:
inv_money_history = pd.DataFrame(inv_money_list, index = inv_money_date) 